In [65]:
import pandas as pd
%matplotlib inline

<h1><left>What are the 5 most popular trips that you would call "commuter trips"?</center></h1>

In order to answer this question, we firstly define the "commuter trips" as ***weekday trips that have start and end time with in 7-9 am or 5-7 pm, and the trip duration is less than 2 hours***. 

Then we need to firstly create a new table, which contains columns of the key of start-end station pair, the name of start station and end station, the start time and end time, and the trip duration. The new table is based on the table of *san_francisco.bikeshare_trips* , and save that as *trip_stations* in BQ. The query code for the new table on the bq web interface is showing below: 

```
#standardSQL
SELECT trip_id, 
    concat(CAST (start_station_id as STRING) , "-", CAST (end_station_id as STRING)) as station_key, 
    concat(CAST (end_station_id as STRING) , "-", CAST (start_station_id as STRING)) as station_key_reverse,
    start_station_name, end_station_name,
    TIMESTAMP_DIFF(end_date, start_date, MINUTE) as trips_duration,
    CAST (start_date as TIME) as start_time,
    CAST (end_date as TIME) as end_time,
    FORMAT_DATE("%A", DATE(start_date)) as start_dow,
    FORMAT_DATE("%A", DATE(end_date)) as end_dow,
    CAST (start_date as DATE) as start_dates,
    CAST (end_date as DATE) as end_dates,
    subscriber_type,
    bike_number
FROM `bigquery-public-data.san_francisco.bikeshare_trips`
```

In [66]:
! bq query --use_legacy_sql=FALSE 'SELECT trip_id, concat(CAST(start_station_id as STRING),"-", CAST(end_station_id as STRING)) as station_key, concat(CAST (end_station_id as STRING) , "-", CAST (start_station_id as STRING)) as station_key_reverse, start_station_name, end_station_name, TIMESTAMP_DIFF(end_date, start_date, MINUTE) as trips_duration, CAST (start_date as TIME) as start_time, CAST (end_date as TIME) as end_time, FORMAT_DATE("%A", DATE(start_date)) as start_dow, FORMAT_DATE("%A", DATE(end_date)) as end_dow, CAST (start_date as DATE) as start_dates, CAST (end_date as DATE) as end_dates, subscriber_type, bike_number FROM `bigquery-public-data.san_francisco.bikeshare_trips` LIMIT 5'

Waiting on bqjob_r1766424725f264c7_00000161cf8e00a0_1 ... (0s) Current status: DONE   
+---------+-------------+---------------------+--------------------+------------------+----------------+------------+----------+-----------+----------+-------------+------------+-----------------+-------------+
| trip_id | station_key | station_key_reverse | start_station_name | end_station_name | trips_duration | start_time | end_time | start_dow | end_dow  | start_dates | end_dates  | subscriber_type | bike_number |
+---------+-------------+---------------------+--------------------+------------------+----------------+------------+----------+-----------+----------+-------------+------------+-----------------+-------------+
|  944732 | 83-83       | 83-83               | Mezes              | Mezes            |             44 |   17:22:00 | 18:06:00 | Thursday  | Thursday |  2015-09-24 | 2015-09-24 | Customer        |         653 |
| 1129385 | 83-83       | 83-83               | Mezes              | 

Since we have the new trip_stations table, we can do the query and check out the top-counted station_key with the definition of "commuter trips". 

- **Firstly, check 5 most popular commuter trips in the morning (7-9 am):**

```
#standardSQL
SELECT station_key, ROUND(AVG(trips_duration),3) as trips_duration_ave, count(trip_id) as trips_am
FROM `silent-card-193103.bike_trips_data.trip_stations`
WHERE trips_duration < 120 
    AND (start_time > TIME (07, 00, 00) AND start_time < TIME (09, 00, 00) AND end_time > TIME (07, 00, 00) AND end_time < TIME (09, 00, 00)) 
    AND (start_dow = "Monday" OR start_dow = "Tuesday" OR start_dow = "Wednesday" OR start_dow = "Thursday" OR start_dow = "Friday")
GROUP BY station_key
ORDER by trips_am DESC
```

In [67]:
! bq query --use_legacy_sql=FALSE 'SELECT station_key, ROUND(AVG(trips_duration),3) as trips_duration_ave, count(trip_id) as trips_am FROM `silent-card-193103.bike_trips_data.trip_stations` WHERE trips_duration < 120 AND (start_time > TIME (07, 00, 00) AND start_time < TIME (09, 00, 00) AND end_time > TIME (07, 00, 00) AND end_time < TIME (09, 00, 00)) AND (start_dow = "Monday" OR start_dow = "Tuesday" OR start_dow = "Wednesday" OR start_dow = "Thursday" OR start_dow = "Friday") GROUP BY station_key ORDER by trips_am DESC LIMIT 5'

Waiting on bqjob_rb38359dac5108d_00000161cf8e1130_1 ... (0s) Current status: DONE   
+-------------+--------------------+----------+
| station_key | trips_duration_ave | trips_am |
+-------------+--------------------+----------+
| 50-61       |              9.668 |     2915 |
| 70-51       |             10.063 |     2362 |
| 70-50       |             11.759 |     2309 |
| 50-60       |              6.807 |     1999 |
| 70-55       |             11.092 |     1938 |
+-------------+--------------------+----------+


   Save the table as *trip_stations_am*. Then, create a new table *trip_stations_am_join* which shows the final result of the top commuter trips by station keys, station names, average trip durations in minutes, and the number of morning trips:

```
#standardSQL
SELECT DISTINCT(`silent-card-193103.bike_trips_data.trip_stations_am`.station_key), 
    `silent-card-193103.bike_trips_data.trip_stations`.station_key_reverse, 
    `silent-card-193103.bike_trips_data.trip_stations_am`.trips_duration_ave, 
    `silent-card-193103.bike_trips_data.trip_stations_am`.trips_am, 
    `silent-card-193103.bike_trips_data.trip_stations`.start_station_name, 
    `silent-card-193103.bike_trips_data.trip_stations`.end_station_name
FROM `silent-card-193103.bike_trips_data.trip_stations_am`
INNER JOIN `silent-card-193103.bike_trips_data.trip_stations` 
    ON `silent-card-193103.bike_trips_data.trip_stations_am`.station_key = `silent-card-193103.bike_trips_data.trip_stations`.station_key
ORDER by trips_am DESC
```

In [68]:
! bq query --use_legacy_sql=FALSE 'SELECT DISTINCT(`silent-card-193103.bike_trips_data.trip_stations_am`.station_key), `silent-card-193103.bike_trips_data.trip_stations`.station_key_reverse, `silent-card-193103.bike_trips_data.trip_stations_am`.trips_duration_ave, `silent-card-193103.bike_trips_data.trip_stations_am`.trips_am, `silent-card-193103.bike_trips_data.trip_stations`.start_station_name, `silent-card-193103.bike_trips_data.trip_stations`.end_station_name FROM `silent-card-193103.bike_trips_data.trip_stations_am` INNER JOIN `silent-card-193103.bike_trips_data.trip_stations` ON `silent-card-193103.bike_trips_data.trip_stations_am`.station_key=`silent-card-193103.bike_trips_data.trip_stations`.station_key ORDER by trips_am DESC LIMIT 5'

Waiting on bqjob_r42b2640bdea19123_00000161cf8e230b_1 ... (1s) Current status: DONE   
+-------------+---------------------+--------------------+----------+------------------------------------------+-----------------------------------------------+
| station_key | station_key_reverse | trips_duration_ave | trips_am |            start_station_name            |               end_station_name                |
+-------------+---------------------+--------------------+----------+------------------------------------------+-----------------------------------------------+
| 50-61       | 61-50               |              9.668 |     2915 | Harry Bridges Plaza (Ferry Building)     | 2nd at Townsend                               |
| 70-51       | 51-70               |             10.063 |     2362 | San Francisco Caltrain (Townsend at 4th) | Embarcadero at Folsom                         |
| 70-50       | 50-70               |             11.759 |     2309 | San Francisco Caltrain (Townsend at 4t

    So, we have the 5 most popular commuter trips in the morning.
    
|order| start_station_name | end_station_name|      
|--|--|------------- |
|1|Harry Bridges Plaza (Ferry Building)    | 2nd at Townsend    |                          
|2|San Francisco Caltrain (Townsend at 4th)| Embarcadero at Folsom  |                      
|3|San Francisco Caltrain (Townsend at 4th)| Harry Bridges Plaza (Ferry Building) |        
|4|Harry Bridges Plaza (Ferry Building)    | Embarcadero at Sansome               |        
|5|San Francisco Caltrain (Townsend at 4th)| Temporary Transbay Terminal (Howard at Beale)|


- **Then, let's check the 5 most popular commuter trips in the afternoon (5-7 pm):**

```
#standardSQL
SELECT station_key, ROUND(AVG(trips_duration),3) as trips_duration_ave, count(trip_id) as trips_pm
FROM `silent-card-193103.bike_trips_data.trip_stations`
WHERE trips_duration < 120 
    AND (start_time > TIME (17, 00, 00) AND start_time < TIME (19, 00, 00) AND end_time > TIME (17, 00, 00) AND end_time < TIME (19, 00, 00)) 
    AND (start_dow = "Monday" OR start_dow = "Tuesday" OR start_dow = "Wednesday" OR start_dow = "Thursday" OR start_dow = "Friday")
GROUP BY station_key
ORDER by trips_pm DESC
```

In [69]:
! bq query --use_legacy_sql=FALSE 'SELECT station_key, ROUND(AVG(trips_duration),3) as trips_duration_ave, count(trip_id) as trips_pm FROM `silent-card-193103.bike_trips_data.trip_stations` WHERE trips_duration < 120 AND (start_time > TIME (17, 00, 00) AND start_time < TIME (19, 00, 00) AND end_time > TIME (17, 00, 00) AND end_time < TIME (19, 00, 00)) AND (start_dow = "Monday" OR start_dow = "Tuesday" OR start_dow = "Wednesday" OR start_dow = "Thursday" OR start_dow = "Friday") GROUP BY station_key ORDER by trips_pm DESC LIMIT 5'

Waiting on bqjob_r31d3dd7a0ee0d500_00000161cf8e398a_1 ... (0s) Current status: DONE   
+-------------+--------------------+----------+
| station_key | trips_duration_ave | trips_pm |
+-------------+--------------------+----------+
| 60-74       |              6.809 |     2618 |
| 64-77       |              6.165 |     2488 |
| 61-50       |              8.211 |     2358 |
| 67-69       |             10.152 |     2217 |
| 65-69       |                4.0 |     2015 |
+-------------+--------------------+----------+


Save the table as *trip_stations_pm*. Then, create a new table *trip_stations_pm_join* which shows the final result of the top commuter trips by station keys, station names, average trip durations in minutes, and the number of morning trips:
```
#standardSQL
SELECT DISTINCT(`silent-card-193103.bike_trips_data.trip_stations_pm`.station_key), 
    `silent-card-193103.bike_trips_data.trip_stations`.station_key_reverse, 
    `silent-card-193103.bike_trips_data.trip_stations_pm`.trips_duration_ave, 
    `silent-card-193103.bike_trips_data.trip_stations_pm`.trips_pm, 
    `silent-card-193103.bike_trips_data.trip_stations`.start_station_name, 
    `silent-card-193103.bike_trips_data.trip_stations`.end_station_name
FROM `silent-card-193103.bike_trips_data.trip_stations_pm`
INNER JOIN `silent-card-193103.bike_trips_data.trip_stations` 
    ON `silent-card-193103.bike_trips_data.trip_stations_pm`.station_key = `silent-card-193103.bike_trips_data.trip_stations`.station_key
ORDER by trips_pm DESC
```

In [70]:
! bq query --use_legacy_sql=FALSE 'SELECT DISTINCT(`silent-card-193103.bike_trips_data.trip_stations_pm`.station_key), `silent-card-193103.bike_trips_data.trip_stations`.station_key_reverse, `silent-card-193103.bike_trips_data.trip_stations_pm`.trips_duration_ave, `silent-card-193103.bike_trips_data.trip_stations_pm`.trips_pm, `silent-card-193103.bike_trips_data.trip_stations`.start_station_name, `silent-card-193103.bike_trips_data.trip_stations`.end_station_name FROM `silent-card-193103.bike_trips_data.trip_stations_pm` INNER JOIN `silent-card-193103.bike_trips_data.trip_stations` ON `silent-card-193103.bike_trips_data.trip_stations_pm`.station_key = `silent-card-193103.bike_trips_data.trip_stations`.station_key ORDER by trips_pm DESC LIMIT 5'

Waiting on bqjob_r116520682ba00a4c_00000161cf8e4b6d_1 ... (0s) Current status: DONE   
+-------------+---------------------+--------------------+----------+------------------------+-----------------------------------------+
| station_key | station_key_reverse | trips_duration_ave | trips_pm |   start_station_name   |            end_station_name             |
+-------------+---------------------+--------------------+----------+------------------------+-----------------------------------------+
| 60-74       | 74-60               |              6.809 |     2618 | Embarcadero at Sansome | Steuart at Market                       |
| 64-77       | 77-64               |              6.165 |     2488 | 2nd at South Park      | Market at Sansome                       |
| 61-50       | 50-61               |              8.211 |     2358 | 2nd at Townsend        | Harry Bridges Plaza (Ferry Building)    |
| 67-69       | 69-67               |             10.152 |     2217 | Market at 10th       

    So, we have the 5 most popular commuter trips in the afternoon.
    
|order| start_station_name | end_station_name|      
|--|--|------------- |
|1|Embarcadero at Sansome | Steuart at Market  |                          
|2|2nd at South Park | Market at Sansome |                      
|3|2nd at Townsend	| Harry Bridges Plaza (Ferry Building) |        
|4|Market at 10th |	San Francisco Caltrain 2 (330 Townsend)	        |        
|5|Townsend at 7th | San Francisco Caltrain 2 (330 Townsend)|



- **Finally, we want to know the 5 most popular commuter trips for the whole day.** 

    Since the start station in the morning become the end station in the afternoon, we need to match the *station_key* in the morning table with the *station_key_reverse* in the afternoon table. 

    We can figure that by the query below:
```
#standardSQL
SELECT DISTINCT(`silent-card-193103.bike_trips_data.trip_stations_am_join`.station_key) as station_key_am, 
    `silent-card-193103.bike_trips_data.trip_stations_am_join`.station_key_reverse as station_key_pm, 
    `silent-card-193103.bike_trips_data.trip_stations_am_join`.trips_duration_ave, 
    `silent-card-193103.bike_trips_data.trip_stations_am_join`.trips_am, 
    `silent-card-193103.bike_trips_data.trip_stations_pm_join`.trips_pm,
    `silent-card-193103.bike_trips_data.trip_stations_am_join`.trips_am + `silent-card-193103.bike_trips_data.trip_stations_pm_join`.trips_pm as trips_total,
    `silent-card-193103.bike_trips_data.trip_stations_am_join`.start_station_name as start_station_name_am, 
    `silent-card-193103.bike_trips_data.trip_stations_am_join`.end_station_name as end_station_name_am
FROM `silent-card-193103.bike_trips_data.trip_stations_am_join`
INNER JOIN `silent-card-193103.bike_trips_data.trip_stations_pm_join` 
    ON `silent-card-193103.bike_trips_data.trip_stations_am_join`.station_key = `silent-card-193103.bike_trips_data.trip_stations_pm_join`.station_key_reverse
ORDER by trips_total DESC
```

In [71]:
! bq query --use_legacy_sql=FALSE 'SELECT DISTINCT(`silent-card-193103.bike_trips_data.trip_stations_am_join`.station_key) as station_key_am, `silent-card-193103.bike_trips_data.trip_stations_am_join`.station_key_reverse as station_key_pm, `silent-card-193103.bike_trips_data.trip_stations_am_join`.trips_duration_ave, `silent-card-193103.bike_trips_data.trip_stations_am_join`.trips_am, `silent-card-193103.bike_trips_data.trip_stations_pm_join`.trips_pm, `silent-card-193103.bike_trips_data.trip_stations_am_join`.trips_am + `silent-card-193103.bike_trips_data.trip_stations_pm_join`.trips_pm as trips_total, `silent-card-193103.bike_trips_data.trip_stations_am_join`.start_station_name as start_station_name_am, `silent-card-193103.bike_trips_data.trip_stations_am_join`.end_station_name as end_station_name_am FROM `silent-card-193103.bike_trips_data.trip_stations_am_join` INNER JOIN `silent-card-193103.bike_trips_data.trip_stations_pm_join` ON `silent-card-193103.bike_trips_data.trip_stations_am_join`.station_key = `silent-card-193103.bike_trips_data.trip_stations_pm_join`.station_key_reverse ORDER by trips_total DESC LIMIT 5'

Waiting on bqjob_r7f400e19dd9c74bc_00000161cf8e5f59_1 ... (0s) Current status: DONE   
+----------------+----------------+--------------------+----------+----------+-------------+------------------------------------------+------------------------+
| station_key_am | station_key_pm | trips_duration_ave | trips_am | trips_pm | trips_total |          start_station_name_am           |  end_station_name_am   |
+----------------+----------------+--------------------+----------+----------+-------------+------------------------------------------+------------------------+
| 50-61          | 61-50          |              9.668 |     2915 |     2358 |        5273 | Harry Bridges Plaza (Ferry Building)     | 2nd at Townsend        |
| 74-60          | 60-74          |              6.873 |     1711 |     2618 |        4329 | Steuart at Market                        | Embarcadero at Sansome |
| 70-51          | 51-70          |             10.063 |     2362 |     1959 |        4321 | San Francisco C

So, we have the 5 most popular commuter trips for the whole day.

|order_total| station_key_am| station_key_pm| trips_duration_ave| trips_am| trips_pm| trips_total| start_station_name_am| end_station_name_am	
|---|---|---|---|---|---|---|---|---
|1| 50-61| 61-50| 9.668| 2915| 2358| 5273| Harry Bridges Plaza (Ferry Building)| 2nd at Townsend	 
|2| 74-60| 60-74| 6.873| 1711| 2618| 4329| Steuart at Market| Embarcadero at Sansome	 
|3| 70-51| 51-70| 10.063| 2362| 1959| 4321| San Francisco Caltrain (Townsend at 4th)| Embarcadero at Folsom 
|4| 69-65| 65-69| 4.175| 1919| 2015| 3934| San Francisco Caltrain 2 (330 Townsend)| Townsend at 7th	 
|5| 70-74| 74-70| 11.263| 1921| 1972| 3893| San Francisco Caltrain (Townsend at 4th)| Steuart at Market

<h1><left>What are your recommendations for offers (justify based on your findings)?</center></h1>

**1. Discount offer for the customer who return bikes to Embarcadero at Sansome and San Francisco Caltrain (Townsend at 4th) from the other stations, during and between two commuter hours on weekdays**

According to the top commuter trips table above, we can see that the number of outbound trips in the afternoon of station Embarcadero at Sansome and San Francisco Caltrain (Townsend at 4th) are much higher than that of inbound trips in the morning, leading to a unbalance bike supplement. In order to full fill the need of the commuters, the company may have to move bike from nearby stations to Embarcadero at Sansome and San Francisco Caltrain (Townsend at 4th), that generate more cost. 


|order_total|  trips_am| trips_pm| trips_total| start_station_name_am| end_station_name_am	
|---|---|---|---|---|---|---|---	 
|2| 1711| 2618| 4329| Steuart at Market| Embarcadero at Sansome	 
|3| 2362| 1959| 4321| San Francisco Caltrain (Townsend at 4th)| Embarcadero at Folsom 
|5| 1921| 1972| 3893| San Francisco Caltrain (Townsend at 4th)| Steuart at Market

So, if we can offer discounts for the travelers return bike to the Embarcadero at Sansome and San Francisco Caltrain (Townsend at 4th)	from the other stations to encourage customers move the bike for us, we can decrease the cost of bike movement and increase profit accordingly. 



**2. Less commuters on Friday, can offer promotions to encourage leisure trip customers. **

Friday has less commuter trips than the other DOW in the morning and the trips in the weekends are also very much different from the weekday commuter trips, in terms of number of trips and trip duration. 

```
#standardSQL
SELECT start_dow, ROUND(AVG(trips_duration),3) as trips_duration_ave, count(trip_id) as trips
FROM `silent-card-193103.bike_trips_data.trip_stations`
WHERE trips_duration < 120 
    AND ((start_time > TIME (07, 00, 00) AND start_time < TIME (09, 00, 00) AND end_time > TIME (07, 00, 00) AND end_time < TIME (09, 00, 00))
    OR(start_time > TIME (17, 00, 00) AND start_time < TIME (19, 00, 00) AND end_time > TIME (17, 00, 00) AND end_time < TIME (19, 00, 00)))
    AND (start_dow = "Monday" OR start_dow = "Tuesday" OR start_dow = "Wednesday" OR start_dow = "Thursday" OR start_dow = "Friday" OR start_dow = "Saturday" OR start_dow = "Sunday")
GROUP BY start_dow
ORDER by trips DESC
```

In [72]:
! bq query --use_legacy_sql=FALSE 'SELECT start_dow, ROUND(AVG(trips_duration),3) as trips_duration_ave, count(trip_id) as trips FROM `silent-card-193103.bike_trips_data.trip_stations` WHERE trips_duration < 120  AND ((start_time > TIME (07, 00, 00) AND start_time < TIME (09, 00, 00) AND end_time > TIME (07, 00, 00) AND end_time < TIME (09, 00, 00)) OR(start_time > TIME (17, 00, 00) AND start_time < TIME (19, 00, 00) AND end_time > TIME (17, 00, 00) AND end_time < TIME (19, 00, 00))) AND (start_dow = "Monday" OR start_dow = "Tuesday" OR start_dow = "Wednesday" OR start_dow = "Thursday" OR start_dow = "Friday" OR start_dow = "Saturday" OR start_dow = "Sunday") GROUP BY start_dow ORDER by trips DESC'

Waiting on bqjob_r1b2410f45aa54563_00000161cf8e6d4b_1 ... (0s) Current status: DONE   
+-----------+--------------------+-------+
| start_dow | trips_duration_ave | trips |
+-----------+--------------------+-------+
| Tuesday   |              9.122 | 77838 |
| Wednesday |              9.133 | 74980 |
| Monday    |              9.056 | 72155 |
| Thursday  |              9.181 | 71135 |
| Friday    |              9.245 | 59989 |
| Saturday  |             13.067 |  8980 |
| Sunday    |             14.094 |  7132 |
+-----------+--------------------+-------+


Then we need to figure out if the subscriber type related with the less commuter trip on Friday.

- **firstly, we can see the number of trips and trip durations by subscriber_type on Friday and on the other day of week:**

```
#standardSQL
SELECT subscriber_type as Friday_subscriber, count(trip_id) as trips, ROUND(AVG(trips_duration),3) as trips_duration_ave
FROM `silent-card-193103.bike_trips_data.trip_stations`
WHERE trips_duration < 120 
    AND ((start_time > TIME (07, 00, 00) AND start_time < TIME (09, 00, 00) AND end_time > TIME (07, 00, 00) AND end_time < TIME (09, 00, 00))
      OR (start_time > TIME (17, 00, 00) AND start_time < TIME (19, 00, 00) AND end_time > TIME (17, 00, 00) AND end_time < TIME (19, 00, 00)))
    AND (start_dow = "Friday")
GROUP BY subscriber_type
```

In [73]:
! bq query --use_legacy_sql=FALSE 'SELECT subscriber_type as Friday_subscriber, count(trip_id) as trips, ROUND(AVG(trips_duration),3) as trips_duration_ave FROM `silent-card-193103.bike_trips_data.trip_stations` WHERE trips_duration < 120  AND ((start_time > TIME (07, 00, 00) AND start_time < TIME (09, 00, 00) AND end_time > TIME (07, 00, 00) AND end_time < TIME (09, 00, 00)) OR (start_time > TIME (17, 00, 00) AND start_time < TIME (19, 00, 00) AND end_time > TIME (17, 00, 00) AND end_time < TIME (19, 00, 00))) AND (start_dow = "Friday") GROUP BY subscriber_type'

Waiting on bqjob_r6757f6aa6a078453_00000161cf8e7a4d_1 ... (0s) Current status: DONE   
+-------------------+-------+--------------------+
| Friday_subscriber | trips | trips_duration_ave |
+-------------------+-------+--------------------+
| Subscriber        | 56764 |              8.832 |
| Customer          |  3225 |             16.523 |
+-------------------+-------+--------------------+


```
#standardSQL
SELECT subscriber_type as otherDOW_subscriber, count(trip_id) as trips, ROUND(AVG(trips_duration),3) as trips_duration_ave
FROM `silent-card-193103.bike_trips_data.trip_stations`
WHERE trips_duration < 120 
    AND ((start_time > TIME (07, 00, 00) AND start_time < TIME (09, 00, 00) AND end_time > TIME (07, 00, 00) AND end_time < TIME (09, 00, 00))
      OR (start_time > TIME (17, 00, 00) AND start_time < TIME (19, 00, 00) AND end_time > TIME (17, 00, 00) AND end_time < TIME (19, 00, 00)))
    AND (start_dow = "Tuesday" OR start_dow = "Wednesday" OR start_dow = "Thursday" OR start_dow = "Monday")
GROUP BY subscriber_type
```

In [74]:
! bq query --use_legacy_sql=FALSE 'SELECT subscriber_type as otherDOW_subscriber, count(trip_id) as trips, ROUND(AVG(trips_duration),3) as trips_duration_ave FROM `silent-card-193103.bike_trips_data.trip_stations` WHERE trips_duration < 120  AND ((start_time > TIME (07, 00, 00) AND start_time < TIME (09, 00, 00) AND end_time > TIME (07, 00, 00) AND end_time < TIME (09, 00, 00)) OR (start_time > TIME (17, 00, 00) AND start_time < TIME (19, 00, 00) AND end_time > TIME (17, 00, 00) AND end_time < TIME (19, 00, 00))) AND (start_dow = "Tuesday" OR start_dow = "Wednesday" OR start_dow = "Thursday" OR start_dow = "Monday") GROUP BY subscriber_type'

Waiting on bqjob_r554a6a2d9838334b_00000161cf8e86bb_1 ... (0s) Current status: DONE   
+---------------------+--------+--------------------+
| otherDOW_subscriber | trips  | trips_duration_ave |
+---------------------+--------+--------------------+
| Subscriber          | 284061 |              8.835 |
| Customer            |  12047 |             15.914 |
+---------------------+--------+--------------------+


The proportion of "Customer" on Friday commuter trips is 5.4%, while the other day of week has less proportion of "Customer", 4.1%. "Customer" is 24-hour or 3-day member, who are more likely to be tourists; and "Subscriber" is annual or 30-day member, who are more likely to be commuters and residents.

The difference in subscriber type shows that because Friday has less commuters by nature than the other day of week, it has less number of trips in the commuter hours. Hence, busy stations in Friday will not be as busy as that of the other day of week, indicating more bikes available for travelers and leisure purpose.

For that reason, Friday can offer similar promotions as Saturday and Sunday for leisure customers who want to rent the bike longer (trip duration longer that 1 day). So, that the utility of the bike can increase, and the revenue can be increased.

